In [40]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import pickle
import os

mypath = os.getcwd() + '/ready'
files = [''.join([mypath, '/', f]) for f in listdir(mypath) if isfile(join(mypath, f)) ]

In [41]:
# create a dataframe for data exploration
import numpy as np

def totuple(a):
    try:
        return tuple(totuple(i) for i in a)
    except TypeError:
        return a

def clean_file_name(f):
    return f.replace(f'{os.getcwd()}/ready/', '').replace('.csv.pkl', '').replace('.', '')

subset_obs = 1000
names = []
              
FLAT_NAMES = ['xCentroid', 'xrms', 'yCentroid', 'yrms', 'uCentroid', 'urms', 'vCentroid', 'vrms', 'Gaussian_Sum']
well_formed_arrays = []
for i, file_name in enumerate(files):
    x = np.load(file_name, allow_pickle=True)[:subset_obs]
    if x.shape[0]: #some are 0
        if x.shape[1] > 1:
            flatten = np.split(x, x.shape[1], axis =1)
            for j, f in enumerate(flatten):
                expand_name = clean_file_name(file_name) + '_'+ FLAT_NAMES[j]
                names.append(expand_name)
#                 print(f"Attribute {expand_name} has shape {.shape}")
                well_formed_arrays.append(np.array(f))
        else:
            names.append(clean_file_name(file_name))
            well_formed_arrays.append(x)

In [42]:
df = pd.DataFrame(np.concatenate(well_formed_arrays, axis = 1), columns = names)
df.head()

,Fwd1Amp,LCamGaussBG_xCentroid,LCamGaussBG_xrms,LCamGaussBG_yCentroid,LCamGaussBG_yrms,LCamGaussBG_uCentroid,LCamGaussBG_urms,LCamGaussBG_vCentroid,LCamGaussBG_vrms,LCamGaussBG_Gaussian_Sum,...,SCam3_Gauss_xrms,SCam3_Gauss_yCentroid,SCam3_Gauss_yrms,SCam3_Gauss_uCentroid,SCam3_Gauss_urms,SCam3_Gauss_vCentroid,SCam3_Gauss_vrms,SCam3_Gauss_Gaussian_Sum,LP_Amp,Fwd2Amp
0,0.022508,32.003670,8.270485,42.817985,8.089002,43.040927,8.848505,46.787501,7.987521,235214.116828,...,32.891862,68.852859,8.837503,375.222468,24.502205,426.628428,24.546571,1.842505e+06,0.366276,0.512519
1,0.022537,31.682232,7.919545,42.790500,8.070923,42.555308,8.244010,46.976442,7.834073,134689.852673,...,27.386855,71.485349,8.987038,383.426830,21.070552,422.756037,20.662912,8.265655e+05,0.366451,0.512598
2,0.022541,32.003254,8.434171,42.783151,8.257484,43.060566,8.999335,46.862217,8.029195,271305.418341,...,34.278664,66.917990,8.998706,380.102540,25.626671,418.882018,24.991122,2.182978e+06,0.366502,0.512604
3,0.022562,31.844320,8.549102,42.984730,8.491756,43.214291,9.033780,47.377427,8.326892,270607.890889,...,34.318775,66.345845,8.596570,368.181755,25.504587,429.979276,25.667503,2.136607e+06,0.366510,0.512576
4,0.022517,31.892984,8.352428,42.924966,8.361304,43.355929,8.901612,47.063289,8.280108,279649.641176,...,33.752881,74.411032,8.878195,389.240224,25.194534,420.522658,25.024092,2.095492e+06,0.366518,0.512594


In [48]:
df.to_pickle('pickled_first_1000_df.pk')